# Update XPT RFID settings

Module interaction:

- Environement selection
- Premise selection
- Device selection
- Update parameters:
- - Tag Population Estimate
- => update device rfid properties
- For each POV
- Update parameters:
- - Tx Power in Dbm
- - Rx Sensitivity in Dbm
- - Rf Time per Ant
- => update pov rfid properties

## Select device to update

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtils

In [ ]:
%%capture
myEnv = chooseEnv.chooseEnv();

In [ ]:
print("Organization: " + myEnv.get('urlDP'))

In [ ]:
%%capture
myEnv = chooseEnv.chooseDevice(myEnv, deviceTypeSelector='XPOINT')
deviceId = myEnv['device']['id']

In [ ]:
print('Device Id: ' + deviceId)

## Update RFID settings

In [ ]:
%%capture
result = deviceUtils.getDeviceById(myEnv, deviceId)

In [ ]:
result

In [ ]:
myForm = uiUtils.inputWithForm('Do you want to update device settings ' + result['device']['name'],['choice Y(yes) or N(no)?'])
choice = myForm.get('choice Y(yes) or N(no)?').upper() == 'Y'

In [ ]:
%%capture
if choice:
    settings = {}
    for mode in result['deviceMode']:
        device = settingsUtils.getRfidSettingsByDeviceId(myEnv, result['device']['id'], mode['id'])
        myForm = uiUtils.inputWithForm('RFID Profile Update for mode ' + mode['id'],['tagPopulationEstimate(integer)'])
        tagPopulationEstimate = myForm.get('tagPopulationEstimate(integer)')
        settings[result['device']['id'] + '_' + mode['id']] = (result['device']['id'], mode['id'], 'tagPopulationEstimate', int(tagPopulationEstimate))

In [ ]:
if choice:
    for s in settings.values():
        settingsUtils.updateDeviceRfidSetting(myEnv, *s)

In [ ]:
for pov in result['devicePov']:
    povId = pov['id']
    myForm = uiUtils.inputWithForm('Do you want to update pov settings ' + pov['name'],['choice Y(yes) or N(no)?'])
    choice = myForm.get('choice Y(yes) or N(no)?').upper() == 'Y'
    if choice:
        mode = 'XPOINT'
        myForm = uiUtils.inputWithForm('POV RFID Profile Update for pov ' + pov['name'] +' and mode ' + mode,['txPowerinDbm(integer)','rxSensitivityinDbm(integer)','rfTimePerAnt(integer)'])
        txPowerinDbm = myForm.get('txPowerinDbm(integer)')
        rxSensitivityinDbm = myForm.get('rxSensitivityinDbm(integer)')
        rfTimePerAnt = myForm.get('rfTimePerAnt(integer)')
        settingsUtils.updatePovRfidSetting(myEnv, povId, mode, 'txPowerinDbm', int(txPowerinDbm))
        settingsUtils.updatePovRfidSetting(myEnv, povId, mode, 'rxSensitivityinDbm', int(rxSensitivityinDbm))
        settingsUtils.updatePovRfidSetting(myEnv, povId, mode, 'rfTimePerAnt', int(rfTimePerAnt))   

## Get RFID settings

In [ ]:
%%capture
result = deviceUtils.getDeviceById(myEnv, deviceId)

In [ ]:
result

In [ ]:
%%capture
settings = {}
for mode in result['deviceMode']:
        settings[result['device']['id'] + '_' + mode['id']] = settingsUtils.getRfidSettingsByDeviceId(myEnv, result['device']['id'], mode['id'])

In [ ]:
settings

In [ ]:
%%capture
settings = {}
for pov in result['devicePov']:
    povId = pov['id']
    for mode in result['deviceMode']:
        settings[povId + '_' + mode['id']] = settingsUtils.getRfidSettingsByPovId(myEnv, povId, mode['id'])

In [ ]:
settings